# Tweet Fetch

This notebook is to show the progress of fetching tweets.

In [1]:
# Initialisation
import twitter
import pandas as pd
import re
import json
import numpy as np

In [2]:
with open('../credential/bearer_token.txt') as f:
    BEARER_TOKEN = f.read()
t_h = twitter.Twitter(
    auth=twitter.OAuth2(bearer_token=BEARER_TOKEN)
    )

In [5]:
# For loop preparation
tweet_df = pd.DataFrame()
max_tweets = 1000
since_id_buff = ''
search_query = 'shang chi'

# Loop. Expect the amount of tweets fetched to exceed the value of max_tweets a bit. Each of the following tweets will be unique.
while len(tweet_df) < max_tweets:
    query = t_h.search.tweets(q=search_query, lang='en', count=100, tweet_mode='extended') if since_id_buff == '' else t_h.search.tweets(q=search_query, lang='en', count=100, tweet_mode='extended', max_id=since_id_buff)
    since_id_buff = re.search(r'max_id=(.+)&q', query['search_metadata']['next_results']).group(1)
    query = pd.DataFrame(query['statuses'])

    # Get full text, put it in a new header
    query_rt = []
    for i in query['retweeted_status']:
        try:
            query_rt.append(i['full_text'])
        except:
            query_rt.append(np.nan)
    # print(query_rt)
    query_text = []
    for i in range(len(query_rt)):
        # print(i, end='\r')
        query_text.append(query_rt[i] if not pd.isnull(query_rt[i]) else query['full_text'].loc[i])

    query['inferred_text'] = query_text

    # Fixing
    tweet_df = tweet_df.append(query).reset_index().drop('index', axis=1) if tweet_df is not None else query

    if len(tweet_df) >= max_tweets:
        tweet_df = tweet_df.drop_duplicates(subset=['inferred_text']).reset_index().drop('index', axis=1)

# tweet_df.to_json('../dataset/trends/'+trend+'.json', orient='index')

# Loop. Expect 25 seconds run for 10 iterations, yielding 1000 tweets.
# for i in range(max_iter):
#     query = t_h.search.tweets(q='shang chi', lang='en', count=100, tweet_mode='extended') if since_id_buff == '' else t_h.search.tweets(q='shang chi', lang='en', count=100, tweet_mode='extended', max_id=since_id_buff)
#     since_id_buff = re.search(r'max_id=(.+)&q', query['search_metadata']['next_results']).group(1)
#     query = pd.DataFrame(query['statuses'])
#     tweet_df = tweet_df.append(query).reset_index().drop('index', axis=1) if tweet_df is not None else query

In [6]:
tweet_df

,created_at,id,id_str,full_text,truncated,display_text_range,entities,metadata,source,in_reply_to_status_id,...,favorited,retweeted,lang,retweeted_status,extended_entities,possibly_sensitive,quoted_status_id,quoted_status_id_str,quoted_status,inferred_text
0,Wed Nov 24 23:51:22 +0000 2021,1463656521913356291,1463656521913356291,@madtruthh you caught me shang chi was t5,False,"[11, 41]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",1.463656e+18,...,False,False,en,NaN,NaN,NaN,NaN,NaN,NaN,@madtruthh you caught me shang chi was t5
1,Wed Nov 24 23:51:02 +0000 2021,1463656435053694982,1463656435053694982,@niik4k Shang chi,False,"[8, 17]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",1.463651e+18,...,False,False,en,NaN,NaN,NaN,NaN,NaN,NaN,@niik4k Shang chi
2,Wed Nov 24 23:50:40 +0000 2021,1463656343978450947,1463656343978450947,@JamalRIP We thought they would do MCU Shang C...,False,"[10, 74]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/#!/download/ipad"" ...",1.463652e+18,...,False,False,en,NaN,NaN,NaN,NaN,NaN,NaN,@JamalRIP We thought they would do MCU Shang C...
3,Wed Nov 24 23:50:37 +0000 2021,1463656330099527681,1463656330099527681,RT @SimuLiu: I’m Shang Chi bitch,False,"[0, 32]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",NaN,...,False,False,en,{'created_at': 'Sun Nov 21 05:20:50 +0000 2021...,NaN,NaN,NaN,NaN,NaN,I’m Shang Chi bitch
4,Wed Nov 24 23:50:33 +0000 2021,1463656314660265988,1463656314660265988,RT @wandasgreys: if you think shang chi isn't ...,False,"[0, 118]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",NaN,...,False,False,en,{'created_at': 'Mon Nov 22 15:41:42 +0000 2021...,"{'media': [{'id': 1462808489634500609, 'id_str...",False,NaN,NaN,NaN,if you think shang chi isn't one of the best m...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1052,Wed Nov 24 04:47:33 +0000 2021,1463368671175266309,1463368671175266309,showing my dad shang chi. it’s going well so far,False,"[0, 48]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",NaN,...,False,False,en,NaN,NaN,NaN,NaN,NaN,NaN,showing my dad shang chi. it’s going well so far
1053,Wed Nov 24 04:47:28 +0000 2021,1463368649159294977,1463368649159294977,Shang Chi in a nutshell https://t.co/8rrum0xT9A,False,"[0, 23]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",NaN,...,False,False,en,NaN,NaN,False,1.462804e+18,1462803714117410826,{'created_at': 'Mon Nov 22 15:22:37 +0000 2021...,Shang Chi in a nutshell https://t.co/8rrum0xT9A
1054,Wed Nov 24 04:44:14 +0000 2021,1463367835476914183,1463367835476914183,RT @motoridersd: This is where I paused Shang ...,False,"[0, 75]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""https://mobile.twitter.com"" rel=""nofo...",NaN,...,False,False,en,{'created_at': 'Wed Nov 24 04:08:01 +0000 2021...,"{'media': [{'id': 1463358715612975109, 'id_str...",False,NaN,NaN,NaN,This is where I paused Shang Chi 😂 https://t.c...
1055,Wed Nov 24 04:42:56 +0000 2021,1463367506622509058,1463367506622509058,@tanyakanrl Shang chi,False,"[12, 21]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",1.463367e+18

In [20]:
# nested = ['entities', 'extended_entities', 'metadata', 'retweeted_status']
#
# for i in nested:
#     exploded = tweet_df.get(i).apply(pd.Series).add_prefix(i+'_')
#     tweet_df = pd.concat([tweet_df.drop(i, axis=1), exploded], axis=1)

In [21]:
# tweet_df.head()

,created_at,id,id_str,full_text,truncated,display_text_range,source,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,...,retweeted_status_place,retweeted_status_possibly_sensitive,retweeted_status_quoted_status,retweeted_status_quoted_status_id,retweeted_status_quoted_status_id_str,retweeted_status_retweet_count,retweeted_status_retweeted,retweeted_status_source,retweeted_status_truncated,retweeted_status_user
0,Wed Nov 17 16:27:15 +0000 2021,1461008038840705024,1461008038840705024,Shang- Chi and The Legend of The Ten Rings is ...,False,"[0, 55]","<a href=""http://twitter.com/download/iphone"" r...",NaN,None,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Wed Nov 17 16:27:10 +0000 2021,1461008018796171265,1461008018796171265,@LOKlUS IM STILL CAUTIOUS ABOUT POSTING SHANG ...,False,"[8, 57]","<a href=""http://twitter.com/download/iphone"" r...",1.460983e+18,1460983219285811202,1.415061e+18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Wed Nov 17 16:26:20 +0000 2021,1461007808095260690,1461007808095260690,I sent my uncles a cute article about the bus ...,False,"[0, 271]","<a href=""http://twitter.com/download/iphone"" r...",NaN,None,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Wed Nov 17 16:26:10 +0000 2021,1461007767947431937,1461007767947431937,@dkcmusic Shang Chi definitely starts slow.,False,"[10, 43]","<a href=""http://twitter.com/download/iphone"" r...",1.460995e+18,1460995462224400388,2.933091e+07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Wed Nov 17 16:26:02 +0000 2021,1461007732480393221,1461007732480393221,@johnsbernardo @IGN No cause if you seen Shang...,False,"[20, 112]","<a href=""http://twitter.com/download/iphone"" r...",1.461004e+18,1461004000229216260,2.372725e+07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# tweet_df.to_excel('../dataset/tweets-raw.xlsx')
# Better: use JSON.
tweet_df.to_json('../dataset/movie-tweets/shang-chi.json', orient='index')